In [ ]:
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import *
import pandas as pd
import os
import csv
from pytrends.request import TrendReq
from scipy.stats import pearsonr
import numpy as np
import re
import pytrends
import bash
import random
import pdb
import calendar
import requests
from requests.exceptions import Timeout
from pytrends.exceptions import ResponseError
from torpy.circuit import  CellTimeoutError

In [2]:
def init():
    #https://pypi.org/project/torpy/
    #pip install torpy
    from torpy import TorClient
    hostname = 'ifconfig.me'  # It's possible use onion hostname here as well
    with TorClient() as tor:
        # Choose random guard node and create 3-hops circuit
        with tor.create_circuit(3) as circuit:
            # Create tor stream to host
            with circuit.create_stream((hostname, 80)) as stream:
                # Now we can communicate with host
                stream.send(b'GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % hostname.encode())
                recv = stream.recv(1024)
    return 0

In [3]:
def column_name_to_mvt_and_remove_zero_values(df, geo_name):
    df.rename(columns={df.columns[0]: f"MVT_GT_{geo_name}" }, inplace = True)
    df = df[df[f"MVT_GT_{geo_name}"] != 0]
    return df

In [4]:
def execute_init():
    while True:
        try:
            init()
            print("init status completed")
            break
        except (TimeoutError, CellTimeoutError, ConnectionResetError) as error:
            print(f"Init Error Occured: {error}")
            print("Let me sleep for 10 seconds")
            print("ZZzzzz...")
            time.sleep(10)
            print("Was a nice sleep, now let me continue...")
            pass
    return 0

In [5]:
def GT_temp_df(start_date, end_date, geo_code, geo_name):
    temp_df = pd.DataFrame()
    while len(temp_df) == 0:
        try:
            pytrends.build_payload(mvt_keyword, cat=0, timeframe=f"{start_date} {end_date}", geo = geo_code)
            temp_df = pytrends.interest_over_time()

        except (ResponseError, Timeout):
            try:
                !pip install pytrends --upgrade
                pass

            except (ResponseError, Timeout) as error:
                print(f"GT Error Occured: {error}")
                print("Let me sleep for 1 day")
                print("ZZzzzz...")
                time.sleep(random.randint(60*60*24, 60*60*24+30,))
                print("Was a nice sleep, now let me continue...")

    print(f"Obtaining GT MVT from {start_date} to {end_date} in {geo_name} metro area")
    time.sleep(random.randint(120, 130))
    temp_df = column_name_to_mvt_and_remove_zero_values(temp_df, geo_name)
    return temp_df

In [6]:
def combine_df_and_temp_df_and_rescale_by_overlap_factors(df, temp_df):
    df = pd.concat([df, temp_df.iloc[:, 0]], axis = 1)
    if len(df.columns) > 1:
        factor_data = df.dropna(how = "any")
        factor_divided = factor_data.iloc[:, 0]/factor_data.iloc[:, 1]
        adjusted_factor = factor_divided.mean()
        df.iloc[:,1] = df.iloc[:,1]*adjusted_factor
        df = df.mean(axis = 1)
    return df

In [7]:
#basic pytrend settings
pytrends = TrendReq()

In [8]:
'''
kw_list = ["car stolen+find stolen car+report police stolen car+insurance car stolen-dream-check",
    "I raped+raped me+raped report police+being raped+been raped - dream - kobe - trump - porn",
    "my stolen+find my stolen+theft report police+someone stolen my - car - heart - dream - number",
    "burglary+home burglarized+my house broken into+my home broken into+burglary report police - dream",
    "my committed a crime+police arrested my+my arrested+my in jail+my in prison - arrested development",
    "I hate family+parents hate me+I hate my parents+I hate school+I hate teacher+I hate dad+I hate mom"]
'''
#years = [i for i in range(2010, 2016)]
#print(years)
#months = [i for i in range(1, 13, 3)] 
#print(months)

'\nkw_list = ["car stolen+find stolen car+report police stolen car+insurance car stolen-dream-check",\n    "I raped+raped me+raped report police+being raped+been raped - dream - kobe - trump - porn",\n    "my stolen+find my stolen+theft report police+someone stolen my - car - heart - dream - number",\n    "burglary+home burglarized+my house broken into+my home broken into+burglary report police - dream",\n    "my committed a crime+police arrested my+my arrested+my in jail+my in prison - arrested development",\n    "I hate family+parents hate me+I hate my parents+I hate school+I hate teacher+I hate dad+I hate mom"]\n'

### GT Keywords

In [9]:
mvt_keyword = ["car stolen+find stolen car+report police stolen car+insurance car stolen-dream-check"] 

### Geo Location and Geo Codes

In [10]:
region_code_dict = {

    "US-NY-501":"New York NY",
    "US-CA-807":"San Francisco-Oakland-San Jose CA",
    "US-CA-803":"Los Angeles CA",
    "US-CA-825":"San Diego CA", 
    "US-CA-807":"Sacramento-Stockton-Modesto CA",
    "US-AZ-753":"Phoenix AZ",
    "US-MI-505":"Detroit MI",
    "US-NV-839":"Las Vegas NV",
    "US-WA-819":"Seattle-Tacoma WA",
    "US-CO-751":"Denver CO",
    "US-PA-504":"Philadelphia PA",
    "US-GA-524":"Atlanta GA",
    "US-IN-602":"Chicago IL",
    "US-NH-506":"Boston MA-Manchester NH",
    "CA-ON":"Ontario"
                    }


### Dates (From, To)

In [11]:
From  = ["2015-01-01", 
         "2015-07-01", 
         "2016-01-01", 
         "2016-07-01",
         "2017-01-01", 
         "2017-07-01",
         "2018-01-01", 
         "2018-07-01",
         "2019-01-01", 
         "2019-07-01",
         "2020-01-01",
         "2020-07-01",
         "2021-01-01",
         "2021-07-01"]

To = ["2015-07-31", 
      "2016-01-31", 
      "2016-07-31", 
      "2017-01-31", 
      "2017-07-31",
      "2018-01-31", 
      "2018-07-31",
      "2019-01-31", 
      "2019-07-31",
      "2020-01-31",
      "2020-07-31",
      "2021-01-31",
      "2021-07-31",
      "2022-01-31"]

## Execute Pytrends to Pull Daily Data from Google Trends

In [12]:

#Get 100 times daily GT MVT data per city
for times in range(0, 100):
    for geo_code, geo_name in zip(region_code_dict.keys(), region_code_dict.values()):
        df = pd.DataFrame()

        for start_date, end_date in zip(From, To):
            #Get daily estimates with adjusted factors [mean(old/new) * new]
            temp_df = GT_temp_df(start_date, end_date, geo_code, geo_name)
            df = combine_df_and_temp_df_and_rescale_by_overlap_factors(df, temp_df)

        df.name = f"MVT_GT_{geo_name}"
        df.to_csv(f"{geo_name}_daily_{start_date}_{end_date}_{pd.Timestamp.now().strftime('%Y%m%d_%H_%M')}.csv")
        print(f"{geo_name}_daily_{start_date}_{end_date}_{pd.Timestamp.now().strftime('%Y%m%d_%H_%M')} file saved")

Obtaining GT MVT from 2015-01-01 to 2015-07-31 in New York NY metro area


KeyboardInterrupt: 

In [ ]:
test.plot(figsize=(20, 12),  kind ='line')

In [ ]:
[print(i/25*23) for i in test2["car stolen+find stolen car+report police stolen car+insurance car stolen-dream-check"]]

In [ ]:
kw_list=['Facebook', 'Apple', 'Amazon', 'Netflix', 'Google']
pytrend = TrendReq()
#search interest per region
#run model for keywords (can also be competitors)
pytrend.build_payload(kw_list, timeframe='today 1-m')

# Interest by Region
regiondf = pytrend.interest_by_region()
#looking at rows where all values are not equal to 0
regiondf = regiondf[(regiondf != 0).all(1)]

#drop all rows that have null values in all columns
regiondf.dropna(how='all',axis=0, inplace=True)

#visualise
regiondf.plot(figsize=(20, 12), y=kw_list, kind ='bar')

In [ ]:
historicaldf = pytrend.get_historical_interest(['Facebook', 'Apple', 'Amazon', 'Netflix', 'Google'], 
                                               year_start=2020, month_start=1, day_start=1, hour_start=0, 
                                               year_end=2020, month_end=12, day_end=31, hour_end=0, 
                                               cat=0, geo='en-US', gprop='', sleep=0)

#visualise
#plot a timeseries chart
historicaldf.plot(figsize=(20, 12))

#plot seperate graphs, using theprovided keywords
historicaldf.plot(subplots=True, figsize=(20, 12))

In [ ]:
def gt_func(start_date_list, end_date_list, resolution, sample_size, path):
#get monthly GT data
    import inflect
    #give ordinal names to numbers
    p = inflect.engine()

    for t in range(sample_size): #sample from GT 100 times
        #time.sleep(random.randint(21600,21668)) # 6 hrs
        print(f"The {p.ordinal(t+1)} pull from GT")
        qt_list = ["Q1","Q2","Q3","Q4"]
        count = 0
        for i, j in zip(start_date_list, end_date_list):
            df = pd.DataFrame()
            #Q1 - Q4
            if count == 4:
                count = 0
            for k in kw_list:
                #q = the name of the quarter
                q = qt_list[count]
                try:
                    pytrends.build_payload(k, cat=0, timeframe=f'{i.strftime("%Y-%m-%d")} {j.strftime("%Y-%m-%d")}',  geo='US', gprop='')
                    df = pd.concat([df, pytrends.interest_by_region(resolution="DMA", inc_low_vol = False, inc_geo_code = False)], axis = 1)
                    time.sleep(random.randint(120, 130))

                except (Timeout, ResponseError) as e:
                        print(f'!!{i.strftime("%Y-%m-%d")} {j.strftime("%Y-%m-%d")}!! ReadTimeoutError')
                        print(f"Whoops! ReadTimeoutError, the {p.ordinal(t+1)} pull aborted, now we take a rest")
                        print(dt.datetime.now().strftime("%Y/%m/%d_%H:%M:%S"))
                        time.sleep(random.randint(86668, 86888)) # 24 hrs
                        print(f"Rest enough, now we continue!")

                        #continue working
                        pytrends.build_payload(k, cat=0, timeframe=f'{i.strftime("%Y-%m-%d")} {j.strftime("%Y-%m-%d")}',  geo='US', gprop='')
                        df = pd.concat([df, pytrends.interest_by_region(resolution="DMA", inc_low_vol=False, inc_geo_code=False)], axis = 1)
                        time.sleep(random.randint(120, 130))

            df.columns = [f"MVT_{i.strftime('%Y')}_{q}", 
                          f"rape_{i.strftime('%Y')}_{q}", 
                          f"larceny_{i.strftime('%Y')}_{q}", 
                          f"burglary_{i.strftime('%Y')}_{q}", 
                          f"known_deliq_{i.strftime('%Y')}_{q}", 
                          f"low_social_cont_{i.strftime('%Y')}_{q}"]
            df.to_csv(f'{path}gt_crime_{i.strftime("%Y")}_{q}_{dt.datetime.now().strftime("%Y%m%d_%H-%M-%S")}.csv')
            count += 1
        print(f"The {p.ordinal(t+1)} pull from GT Done!!")
        print(f'Time finished: {dt.datetime.now().strftime("%Y/%m/%d, %H:%M")}')
        time.sleep(random.randint(10800,10868)) # 3 hrs


    print(f"Congratulations, Well Done!")